In [1]:
import numpy as np
import hashlib
import math
import pandas as pd 
import matplotlib.pyplot as plt
import random
import re
from nltk.tokenize import WordPunctTokenizer

from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics.pairwise import cosine_similarity

import spacy

In [2]:
df = pd.read_csv('datasets/spam_cleaned.tsv', sep='\t')

simple_tokenizer = lambda doc: " ".join(WordPunctTokenizer().tokenize(doc))
df['email'] = df['email'].map(simple_tokenizer)

X_train, X_test, y_train, y_test = train_test_split(df['email'], df['label'], test_size=0.3, random_state=42)

In [9]:
y_train.value_counts()

ham     3372
spam     528
Name: label, dtype: int64

In [10]:
df.head()

,email,label
0,"Go until jurong point , crazy .. Available onl...",ham
1,Ok lar ... Joking wif u oni ...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor ... U c already then sa...,ham
4,"Nah I don ' t think he goes to usf , he lives ...",ham


## Q1.

The year is 1998 and your hotmail account is starting to fill up with garbage! Luckily you know some NLP and have a handy dataset to train a spam/not spam classifier with. (But beware, the dataset is very imbalanced.) To start, you'll train a baseline classifier using features from TFIDF.


### Q1.a)

Don't forget your computer is also from 1998 and your computing power is pretty bad, so we'll have to limit this classifier to only using 50 features.
Implement a function that returns the fitted vectorizer, and precision and recall, respectively, on the spam class from a K Nearest Neighbors classifier using default TFIDF features (except max_features=50).

In [36]:
y_train.values == 'spam'

array([ True, False, False, ..., False, False, False])

In [4]:
def get_precision_recall_of_spam_tfidf(X_train, y_train, X_test, y_test):
    '''Returns a fitted TfidfVectorizer and the test precision and recall on the 'spam' class
       from a KNeighborsClassifier trained on the inputted train data
    
    Parameters:
        X_train (Series): Text data for training
        y_train (Series): Labels corresponding to X_train
        X_test (Series): Text data for testing
        y_test (Series): Labels corresponding to X_test

    Returns:
        vectorizer (TfidfVectorizer): TfidfVectorizer with max_features == 50, fitted to X_train
        precision (float): The precision score of the spam class on the test data from a
                           KNeighborsClassifier fitted to the vectorized training data
        recall (float): The recall score of the spam class on the test data from a
                        KNeighborsClassifier fitted to the vectorized training data
    '''
    
    # YOUR CODE HERE
    vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=50, stop_words=None)
    vectorizer.fit(X_train)
    
    vect_X_train = vectorizer.transform(X_train)
    vect_X_test = vectorizer.transform(X_test)
    
    vect_y_train = y_train.values
    vect_y_test = y_test.values
    
    clf =  KNeighborsClassifier()
    clf.fit(vect_X_train, vect_y_train)

    pred = clf.predict(vect_X_test)
    
    precision = precision_score(vect_y_test=='spam', pred=='spam')
    recall = recall_score(vect_y_test=='spam', pred=='spam')
        
    return vectorizer, precision, recall

In [6]:
vectorizer, precision, recall = get_precision_recall_of_spam_tfidf(X_train, y_train, X_test, y_test)

In [12]:
vectorizer.vocabulary_

{'to': 40,
 'and': 1,
 'the': 37,
 'with': 47,
 'go': 13,
 'from': 11,
 'out': 34,
 'just': 21,
 'is': 19,
 'for': 9,
 'do': 8,
 'you': 48,
 'get': 12,
 'your': 49,
 'call': 6,
 'me': 25,
 'know': 22,
 'or': 33,
 'if': 17,
 'have': 15,
 'at': 3,
 'my': 26,
 'lt': 24,
 'gt': 14,
 'when': 45,
 'can': 7,
 'that': 36,
 'on': 32,
 'no': 27,
 'now': 29,
 'it': 20,
 'of': 30,
 'ok': 31,
 'then': 38,
 'in': 18,
 'so': 35,
 'we': 43,
 'are': 2,
 'll': 23,
 'how': 16,
 'will': 46,
 'this': 39,
 'what': 44,
 'be': 4,
 'not': 28,
 'all': 0,
 'but': 5,
 'ur': 42,
 'free': 10,
 'up': 41}

In [63]:
sorted(vectorizer.vocabulary_.keys(), key = vectorizer.vocabulary_.get)

['all',
 'and',
 'are',
 'at',
 'be',
 'but',
 'call',
 'can',
 'do',
 'for',
 'free',
 'from',
 'get',
 'go',
 'gt',
 'have',
 'how',
 'if',
 'in',
 'is',
 'it',
 'just',
 'know',
 'll',
 'lt',
 'me',
 'my',
 'no',
 'not',
 'now',
 'of',
 'ok',
 'on',
 'or',
 'out',
 'so',
 'that',
 'the',
 'then',
 'this',
 'to',
 'up',
 'ur',
 'we',
 'what',
 'when',
 'will',
 'with',
 'you',
 'your']

In [60]:
vectorizer.idf_

array([4.06498152, 2.97741907, 3.50066721, 3.65951641, 3.78419128,
       3.59223441, 3.35009436, 3.63419861, 3.78834929, 3.16642961,
       4.13908949, 4.01149284, 3.71216015, 4.02724119, 4.06498152,
       3.36635488, 3.96072051, 3.81366709, 2.91288055, 3.003687  ,
       3.26017502, 3.73955912, 4.12149373, 4.09850421, 4.05405245,
       3.06847904, 3.250395  , 3.82225084, 3.63419861, 3.42834655,
       3.3554852 , 3.98072118, 3.43417747, 3.64858734, 4.14502423,
       3.53888843, 3.41391629, 2.689737  , 4.15700042, 3.88909086,
       2.15829209, 3.93626942, 3.86631083, 3.75555946, 3.94111204,
       4.00629802, 3.73955912, 3.81366709, 2.26956574, 3.21925475])

In [68]:
type(vectorizer.get_feature_names())

list

In [70]:
len(vectorizer.vocabulary_)

50

In [43]:
recall

0.6210045662100456

In [44]:
vectorizer, precision, recall = get_precision_recall_of_spam_tfidf(X_train, y_train, X_test, y_test)

assert math.isclose(precision, 0.8395061728395061)
assert math.isclose(recall, 0.6210045662100456)

### Q1.b)

Let's see what we can do to get both precision and recall up. We don't want to be bombarded with spam but we also don't want to miss too many real emails.

First, look at the selected features of the fitted tfidf vectorizer, sorted by their inverse document frequency (in descending order). 

In [65]:
def get_ngrams_sorted_by_idf(fitted_vectorizer):
    '''Returns the features of a fitted TfidfVectorizer ordered by their idf score
    
    Parameters:
        fitted_vectorizer (TfidfVectorizer): A fitted TfidfVectorizer
    
    Returns:
        ngrams_sorted (list): The features of fitted_vectorizer sorted in ascending order
                              by their idf score
    '''
    # YOUR CODE HERE
    features_idf_dict =  dict(zip(fitted_vectorizer.get_feature_names(), fitted_vectorizer.idf_))
    ngrams_sorted = sorted(features_idf_dict.keys(), key = features_idf_dict.get, reverse=True)
    
    return ngrams_sorted

In [66]:
assert(hashlib.sha256(get_ngrams_sorted_by_idf(vectorizer)[5].encode()).hexdigest() == "5ef5ef0364b6939c4ca61f34b393f7b368d1be8619647aaf83d5b395919ab629")
assert(hashlib.sha256(get_ngrams_sorted_by_idf(vectorizer)[48].encode()).hexdigest() == "bb0347a468d97e98a9c00e37cebec1ab930f6f1221cae0f1fbb92b07e1900ba2")

In [19]:
features_idf_dict =  dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
sorted_dict = sorted(features_idf_dict.items(), key = lambda item: item[1], reverse=True)
sorted_dict[:5]

[('then', 4.157000421150114),
 ('out', 4.145024230103397),
 ('free', 4.139089494583583),
 ('know', 4.121493732693203),
 ('ll', 4.098504214468505)]

In [22]:
features_idf_series = pd.Series(vectorizer.idf_, index=vectorizer.get_feature_names())
features_idf_series.sort_values(ascending=False)[:5]

then    4.157000
out     4.145024
free    4.139089
know    4.121494
ll      4.098504
dtype: float64

In [67]:
# show most specifc 30 features of the 50 selected features
get_ngrams_sorted_by_idf(vectorizer)[:30]

['then',
 'out',
 'free',
 'know',
 'll',
 'all',
 'gt',
 'lt',
 'go',
 'from',
 'when',
 'ok',
 'how',
 'what',
 'up',
 'this',
 'ur',
 'no',
 'if',
 'with',
 'do',
 'be',
 'we',
 'just',
 'will',
 'get',
 'at',
 'or',
 'can',
 'not']

### Q1.c)

Let's also check the number of features:

In [71]:
def get_number_of_features(fitted_vectorizer):
    '''Returns the number of features of a fitted TfidfVectorizer
    
    Parameters:
        fitted_vectorizer (TfidfVectorizer): A fitted TfidfVectorizer
    
    Returns:
        vocab_size (int): The number of features of fitted_vectorizer 
    '''
    # YOUR CODE HERE
    vocab_size = len(fitted_vectorizer.vocabulary_)
    return vocab_size

In [72]:
num_features = get_number_of_features(vectorizer)

assert num_features == 50

### Q1.d)

Hmm, looks like we might need to do some feature selection!

To try something simple, reimplement the KNN function from 1a, but avoiding English stopwords.

In [73]:
def get_precision_recall_of_spam_tfidf_no_stopwords(X_train, y_train, X_test, y_test):
    '''Returns a fitted TfidfVectorizer with English stopwords removed and the test precision
       and recall on the 'spam' class from a KNeighborsClassifier trained on the inputted train data
    
    Parameters:
        X_train (Series): Text data for training
        y_train (Series): Labels corresponding to X_train
        X_test (Series): Text data for testing
        y_test (Series): Labels corresponding to X_test

    Returns:
        vectorizer (TfidfVectorizer): TfidfVectorizer with max_features=50 and stopwords='english',
                                      fitted to X_train
        precision (float): The precision score of the spam class on the test data from a
                           KNeighborsClassifier fitted to the vectorized training data
        recall (float): The recall score of the spam class on the test data from a
                        KNeighborsClassifier fitted to the vectorized training data
    '''
    # YOUR CODE HERE
    vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=50, stop_words='english')
    vectorizer.fit(X_train)
    
    vect_X_train = vectorizer.transform(X_train)
    vect_X_test = vectorizer.transform(X_test)
    
    vect_y_train = y_train.values
    vect_y_test = y_test.values
    
    clf =  KNeighborsClassifier()
    clf.fit(vect_X_train, vect_y_train)

    pred = clf.predict(vect_X_test)
    
    precision = precision_score(vect_y_test=='spam', pred=='spam')
    recall = recall_score(vect_y_test=='spam', pred=='spam')
    
    return vectorizer, precision, recall

In [74]:
vectorizer_stopwords, precision_stopwords, recall_stopwords = \
                    get_precision_recall_of_spam_tfidf_no_stopwords(X_train, y_train, X_test, y_test)

assert math.isclose(precision_stopwords, 0.8636363636363636)
assert math.isclose(recall_stopwords, 0.6073059360730594)

In [75]:
# look at most specific features again
get_ngrams_sorted_by_idf(vectorizer_stopwords)[:20]

['happy',
 'way',
 'night',
 'won',
 'pls',
 'week',
 'yes',
 'dear',
 'did',
 'claim',
 'msg',
 'ì_',
 'hey',
 'hi',
 'tell',
 'later',
 'phone',
 'dont',
 'think',
 'mobile']

## Q2.

Not bad, not great, after some simple feature selection we have better precision but slightly worse recall. Still you are confident you can push it a bit further.

Repeat Q1, but this time, in addition to stopwords, use the chi-squared method to get the relevant 50 features, and output the fitted vectorizer, fitted SelectKBest, precision, and recall:

In [76]:
def get_precision_recall_of_spam_chi_squared(X_train, y_train, X_test, y_test):
    '''Returns a fitted TfidfVectorizer, a fitted SelectKBest and the test precision and recall on
       the 'spam' class from a KNeighborsClassifier trained on the inputted train data
    
    Parameters:
        X_train (Series): Text data for training
        y_train (Series): Labels corresponding to X_train
        X_test (Series): Text data for testing
        y_test (Series): Labels corresponding to X_test

    Returns:
        vectorizer (TfidfVectorizer): TfidfVectorizer with  stopwords='english', fitted to X_train
        ch2 (SelectKBest): SelectKBest with score function chi2 and k=50, fitted to vectorized X_train
        precision (float): The precision score of the spam class on the test data from a
                           KNeighborsClassifier fitted to the feature-selected training data
        recall (float): The recall score of the spam class on the test data from a KNeighborsClassifier
                        fitted to the feature-selected training data
    '''
    # YOUR CODE HERE
    vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=None, stop_words='english')
    vectorizer.fit(X_train)
    
    X_train_vec = vectorizer.transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    y_train_vec = y_train.values
    y_test_vec = y_test.values
    
    ch2 = SelectKBest(chi2, k=50)
    ch2.fit(X_train_vec, y_train_vec)
    X_train_chi = ch2.transform(X_train_vec)
    X_test_chi = ch2.transform(X_test_vec)

       
    clf =  KNeighborsClassifier()
    clf.fit(X_train_chi, y_train_vec)

    y_pred = clf.predict(X_test_chi)
    
    precision = precision_score(y_test_vec=='spam', y_pred=='spam')
    recall = recall_score(y_test_vec=='spam', y_pred=='spam')
    
    return vectorizer, ch2, precision, recall

In [77]:
vectorizer_ch2, ch2, precision_ch2, recall_ch2 = \
                    get_precision_recall_of_spam_chi_squared(X_train, y_train, X_test, y_test)

assert math.isclose(precision_ch2, 0.9101123595505618)
assert math.isclose(recall_ch2, 0.7397260273972602)

In [78]:
# Again, we'll check the most important features
most_important_features = [vectorizer_ch2.get_feature_names()[i] for i in ch2.get_support(indices=True)]
print(most_important_features)

['000', '100', '1000', '10p', '150p', '150ppm', '16', '18', '50', '500', '5000', 'apply', 'award', 'awarded', 'camera', 'cash', 'claim', 'collection', 'com', 'contact', 'cs', 'draw', 'entry', 'free', 'guaranteed', 'landline', 'latest', 'line', 'mobile', 'national', 'nokia', 'po', 'prize', 'rate', 'receive', 'reply', 'ringtone', 'service', 'stop', 'text', 'tone', 'tones', 'txt', 'uk', 'urgent', 'video', 'weekly', 'win', 'won', 'www']


## Q3.

Chi-squared feature selection helped get both metrics up a little! 

You now feel confident enough to verify if these features are in fact meaningful for classifying your data, so you decide to plot them.

In [79]:
sub_punct = lambda s: re.sub(r'\s+\W*[a-z]?\W*\s+', ' ', s)

In [80]:
for feature in most_important_features:
    print('Documents that contains the word(s) "%s"' % feature)
    print('----')
    docs = X_train.apply(sub_punct).str.lower().str.contains(feature)
    print(str(y_train[docs].value_counts()) + '\n\n')

Documents that contains the word(s) "000"
----
spam    115
ham       3
Name: label, dtype: int64


Documents that contains the word(s) "100"
----
spam    71
ham      2
Name: label, dtype: int64


Documents that contains the word(s) "1000"
----
spam    35
ham      1
Name: label, dtype: int64


Documents that contains the word(s) "10p"
----
spam    23
Name: label, dtype: int64


Documents that contains the word(s) "150p"
----
spam    102
Name: label, dtype: int64


Documents that contains the word(s) "150ppm"
----
spam    35
Name: label, dtype: int64


Documents that contains the word(s) "16"
----
spam    66
ham      2
Name: label, dtype: int64


Documents that contains the word(s) "18"
----
spam    91
ham      1
Name: label, dtype: int64


Documents that contains the word(s) "50"
----
spam    255
ham       4
Name: label, dtype: int64


Documents that contains the word(s) "500"
----
spam    65
Name: label, dtype: int64


Documents that contains the word(s) "5000"
----
spam    31
Name: la

## Q4.

You now want dimensionality reduction techniques like SVD and PCA. 

In [55]:
# set a seed for random
seed = 1234

### Q4.a)

Transform the training data with tfidf avoiding stopwords, and calculate the total variance 

In [46]:
# vectorizer = 
# X_train_vec = 
# X_test_vec = 
# total_variance = 

# YOUR CODE HERE
vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=None, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
total_variance = np.var(X_train_vec.toarray(), axis=0).sum()
total_variance

#dense_X_train = X_train.toarray()
#data_var = np.var(dense_X_train, axis=0).sum()

0.9897797614643888

In [34]:
a = np.array([[1, 2], [3, 4]])
np.var(a)

1.25

In [38]:
print((1-2.5)**2)
print((2-2.5)**2)
print((3-2.5)**2)
print((4-2.5)**2)
print(5/4)

2.25
0.25
0.25
2.25
1.25


In [36]:
(a-a.mean())**2

array([[2.25, 0.25],
       [0.25, 2.25]])

In [41]:
var = np.mean((a-a.mean())**2)
var

1.25

In [47]:
assert math.isclose(total_variance, 0.9897797614643888)

### Q4.b)

Write a function that fits sklearn's TruncatedSVD with 50 components to the tfidf data, trains a KNN classifier, and outputs the total explained variance, precision, and recall

In [76]:
def get_precision_recall_of_svd(X_train_vec, y_train, X_test_vec, y_test):
    '''Returns the total explained variance of SVD fitted to the Tfidf-vectorized input data,
       and the test precision and recall on the 'spam' class from a KNeighborsClassifier
       trained on the dimensionality-reduced train data
    
    Parameters:
        X_train_vec (Series): Text data for training vectorized by Tfidf
        y_train (Series): Labels corresponding to X_train_vec
        X_test_vec (Series): Text data for testing vectorized by Tfidf
        y_test (Series): Labels corresponding to X_test_vec

    Returns:
        explained variance (float): The total explained variance of a fitted TruncatedSVD
                                    with n_components=50 and random_state=seed
        precision (float): The precision score of the spam class on the test data from a
                           KNeighborsClassifier fitted to the SVD'd training data
        recall (float): The recall score of the spam class on the test data from a
                        KNeighborsClassifier fitted to the SVD'd training data
    '''
    # YOUR CODE HERE
        
    y_train = y_train.values
    y_test = y_test.values
    
    svd = TruncatedSVD(n_components=50, random_state=seed)
    svd.fit(X_train_vec)
    
    X_train_svd = svd.transform(X_train_vec)
    X_test_svd =  svd.transform(X_test_vec)
    
    explained_variance = svd.explained_variance_.sum()
           
    clf =  KNeighborsClassifier()
    clf.fit(X_train_svd, y_train)

    pred = clf.predict(X_test_svd)
    
    precision = precision_score(y_test=='spam', pred=='spam')
    recall = recall_score(y_test=='spam', pred=='spam')
        
    
    return explained_variance, precision, recall#, svd

In [77]:
var, precision_svd, recall_svd = \
                    get_precision_recall_of_svd(X_train_vec, y_train, X_test_vec, y_test)

assert math.isclose(var, 0.16416346460759212)
assert math.isclose(precision_svd, 0.9109947643979057)
assert math.isclose(recall_svd, 0.7945205479452054)

## Q5

SVD got us a few points higher in recall than chi-squared, nice! Remember now some of the pros and cons of SVD and other dimensionality reduction techniques. While SVD performed better, likely because in theory it can capture information from more than 50 ngrams (whereas in chi-squared the number of features was in a direct correspondence with the number of ngrams), with SVD we lose interpretability and can't examine the most important features!

Now let's try PCA.

Write a function that fits sklearn's PCA with 50 components to the tfidf data from Q4, trains a KNN classifier, and outputs the total explained variance, precision, and recall.

In [ ]:
def get_precision_recall_of_pca(X_train_vec, y_train, X_test_vec, y_test):
    '''Returns the total explained variance of PCA fitted to the Tfidf-vectorized input data,
       and the test precision and recall on the 'spam' class from a KNeighborsClassifier
       trained on the dimensionality-reduced train data
    
    Parameters:
        X_train_vec (Series): Text data for training vectorized by Tfidf
        y_train (Series): Labels corresponding to X_train_vec
        X_test_vec (Series): Text data for testing vectorized by Tfidf
        y_test (Series): Labels corresponding to X_test_vec

    Returns:
        explained variance (float): The total explained variance of a fitted PCA
                                    with n_components=50 and random_state=seed
        precision (float): The precision score of the spam class on the test data from a
                           KNeighborsClassifier fitted to the PCA'd training data
        recall (float): The recall score of the spam class on the test data from a
                        KNeighborsClassifier fitted to the PCA'd training data
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return explained_variance, precision, recall

In [ ]:
var_pca, precision_pca, recall_pca = \
                    get_precision_recall_of_pca(X_train_vec, y_train, X_test_vec, y_test)

assert math.isclose(var_pca, 0.1658261296302505)
assert math.isclose(precision_pca, 0.9067357512953368)
assert math.isclose(recall_pca, 0.7990867579908676)

## Q6.

Now we'll change gears a bit and look into word vectors. In Learning Notebook 3 we told you that word vectors can be visualized after being projected into 2D space, and we showed you this diagram:

<img src="./media/word-vectors-projection.png" width="600">

Now we'll try to combine what you've learned about word embeddings and PCA to make our own visualization!

In [ ]:
# load word embeddings

nlp = spacy.load('en_core_web_md')

### Q6.a)

First, to get comfortable with spacy, get the vector for the word "tree" and return the sum of its elements

In [ ]:
# sum_tree = 

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert math.isclose(sum_tree, 2.3972085, abs_tol=0.00001)

### Q6.b) 

Next, write a function that does a fit_transform of sklearn's PCA to a given set of word vectors, using the correct number of components. The function should return the reduced dimension word vectors.

In [ ]:
def reduce_word_vecs(vectors):
    '''Returns PCA-reduced word vectors of the input vectors
    
    Parameters:
        vectors (np.array): Word vectors to be reduced

    Returns:
        reduced_vecs (np.array): Word vectors reduced to the number of dimensions
                                 suitable for plotting with random_state=seed
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return reduced_vecs

In [ ]:
test_vectors = np.array([[0.1, 0.2, 0.3, 0.4], [0.3, 0.5, 0.1, 0.7], [0.8, 0.6, 0.2, 0.4]])
reduced_vecs = reduce_word_vecs(test_vectors)

assert reduced_vecs.shape == (3,2)
assert math.isclose(reduced_vecs[1][1], 0.24736592153367926)
assert math.isclose(reduced_vecs[2][0], 0.43388622222454437)

Now, we'll create an array of ~100,000 of spacy's word vectors and use your PCA function to reduce them. If you're curious about using the full amount of word vectors, just remove the `if` statement in the below function, but beware it will use a lot of memory!

We'll also set a list of words that we'll plot later and force our set of vectors to include these.

In [ ]:
words_to_plot = ['banana', 'pineapple', 'mango', 'red', 'blue', 'yellow', 'woman', 'man', 'child', 'playing',
                 'reading', 'studying', 'nintendo', 'sony', 'xbox', 'sad', 'angry', 'bored']

In [ ]:
# Caution: if you run this cell more than once, you may get different words than the ones
# expected for the rest of the exercises. If that happens, restart the kernel and try again

random.seed(123)

full_vocab_vecs = []
vocab_strings = []
for tok in list(nlp.vocab.strings):
    if tok in words_to_plot or random.random() < 0.07:
        full_vocab_vecs.append(nlp.vocab.get_vector(tok))
        vocab_strings.append(tok)

vocab_array = np.array(full_vocab_vecs)

In [ ]:
print('Word vectors shape pre-PCA: {}'.format(vocab_array.shape))

full_vocab_reduced = reduce_word_vecs(vocab_array)

print('Word vectors shape after PCA: {}'.format(full_vocab_reduced.shape))

In [ ]:
assert math.isclose(full_vocab_reduced[100][0], -3.0945618, abs_tol=0.00001)
assert math.isclose(full_vocab_reduced[9999][0], -0.94902855, abs_tol=0.00001)

### Q6.c)

Time to plot! For this, we'll limit the visualized words to a small subset.

In [ ]:
coords = []
for word in words_to_plot:
    idx = vocab_strings.index(word)
    coords.append(full_vocab_reduced[idx])

coords_array = np.array(coords)

In [ ]:
plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

plt.xlim(min([x for x in coords_array[:,0]]), max([x for x in coords_array[:,0]]))
plt.ylim(min([y for y in coords_array[:,1]]), max([y for y in coords_array[:,1]]))
plt.scatter(coords_array[:,0], coords_array[:,1])

for item, x, y in zip(words_to_plot, coords_array[:,0], coords_array[:,1]):
    plt.annotate(item, xy=(x, y), xytext=(-2, 2), textcoords='offset points', 
                 ha='right', va='bottom', color='purple', fontsize=14 )
plt.show()

The PCA seems to have worked! In the diagram we can see similar types of words closer together. But of course, take these visualizations with a grain of salt because it is practically impossible to preserve all distances in a high dimensional space in just 2 dimensions.

### Q6.d) 

As a final exercise, we'll look at some word similarities.

Write a function that returns the next closest word in terms of cosine similarity to a given word. If there are multiple words with the same highest similarity, return all of them.

Hint: you can use the already-imported `cosine_similarity` function from sklearn to compute cosine similarity.

In [ ]:
def closest_word(input_word, words_in_vocab, word_vectors):
    '''Returns a list of the closest word or words to the input word, based on cosine similarities
       of the word vectors given
    
    Parameters:
        input_word (string): Search for the closest words to this word
        words_in_vocab (list): Vocabulary associated with the vectors in word_vectors
        word_vectors (np.array): Word vectors associated with the strings in words_in_vocab

    Returns:
        closest_words_list (list): List of strings containing the closest word or words to
                                   input_word, based on cosine similarities of the word_vectors
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return closest_words_list

In [ ]:
assert(hashlib.sha256(closest_word('nintendo', words_to_plot, coords_array)[0].encode()).hexdigest() == "f711da60664c04c146d7a47b722c38a8d0bf46c3f52c2084c5c8d1cb78138e73")
assert(hashlib.sha256(closest_word('playing', words_to_plot, coords_array)[0].encode()).hexdigest() == "435c149cbc6a5e5cc373cd33347d4c336a22160e06b7df61092b66e56f4d55ec")
assert(hashlib.sha256(closest_word('pineapple', words_to_plot, coords_array)[0].encode()).hexdigest() == "6815f3c300383519de8e437497e2c3e97852fe8d717a5419d5aafb00cb43c494")